In [ ]:
import numpy as np                     # Llibreria matemÃ tica
import pandas as pd
import matplotlib.pyplot as plt        # Per mostrar plots
import sklearn                         # Llibreia de DM
import sklearn.datasets as ds            # Per carregar mÃ©s facilment el dataset digits
import sklearn.model_selection as cv    # Pel Cross-validation
import sklearn.neighbors as nb           # Per fer servir el knn
%matplotlib inline   

def load_data():
  global df
  from google.colab import drive
  drive.mount('/content/drive')
  df=pd.read_csv('/content/drive/MyDrive/MD/Laboratori/Practica 2/data/smoking_preprocessed.csv')

**LOAD DATA**

In [ ]:
load_data()
df.head()

In [ ]:
# Divide data between all dataset without smoking and smoking label

X = df.drop('smoking', axis=1)
y = df.smoking
(X_train, X_test,  y_train, y_test) = cv.train_test_split(X, y, test_size=.3, random_state=1)

**DECISION TREES**

In [ ]:
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import  ConfusionMatrixDisplay, RocCurveDisplay
from statsmodels.stats.proportion import proportion_confint

clf = tree.DecisionTreeClassifier(criterion='entropy', min_samples_split=2,min_impurity_decrease=0.02)

pred = clf.fit(X_train, y_train).predict(X_test)
fig, axes = plt.subplots(1,2,figsize=(15,5))
ax = axes.reshape(-1)[0]
ConfusionMatrixDisplay.from_estimator(clf, X_test, y_test, display_labels=["no_smoking", "smoking"], ax=ax)

# La curva ROC muestra la relación entre la tasa de verdaderos positivos (TPR, True Positive Rate) en el eje y y la tasa de falsos positivos (FPR, False Positive Rate) en el eje x. 
ax = axes.reshape(-1)[1]
RocCurveDisplay.from_estimator(clf, X_test, y_test, ax=ax, pos_label=1)

# Obtain accuracy score of learned classifier on test data
print(clf.score(X_test, y_test))
print(confusion_matrix(y_test, pred))
print()
print("Accuracy:", sklearn.metrics.accuracy_score(y_test, pred))
print()
print(sklearn.metrics.classification_report(y_test, pred))
epsilon = sklearn.metrics.accuracy_score(y_test, pred)
proportion_confint(count=epsilon*X_test.shape[0], nobs=X_test.shape[0], alpha=0.05, method='normal')

fig = plt.gcf()
fig.set_dpi(150)

tree.plot_tree(clf, filled=True,rounded=True)
fig.savefig('tree.png')
plt.show()

Find best params

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'criterion': ['entropy','gini'],
    'splitter': ['best','random'],
    'min_impurity_decrease': list(np.linspace(0,0.5,21)),
    'min_samples_split':list(range(2,20,11))
    }
clf = GridSearchCV(tree.DecisionTreeClassifier(criterion='entropy'), param_grid=params,cv=10,n_jobs=-1)  # If cv is integer, by default is Stratifyed 
clf.fit(X_train, y_train)
print("Best Params =",clf.best_params_, "Accuracy=", clf.best_score_)
  

Try classification with best params

In [ ]:
clf=tree.DecisionTreeClassifier(criterion=clf.best_params_['criterion'], splitter=clf.best_params_['splitter'], min_samples_split=clf.best_params_['min_samples_split'],min_impurity_decrease=clf.best_params_['min_impurity_decrease'])
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)

# Obtain accuracy score of learned classifier on test data
print(clf.score(X_test, y_test))
print(confusion_matrix(y_test, pred))
print()
print("Accuracy:", sklearn.metrics.accuracy_score(y_test, pred))
print()
print(sklearn.metrics.classification_report(y_test, pred))
epsilon = sklearn.metrics.accuracy_score(y_test, pred)
print("Interval of confidence:", proportion_confint(count=epsilon*X_test.shape[0], nobs=X_test.shape[0], alpha=0.05, method='normal'))


In [ ]:
fig = plt.gcf()
fig.set_dpi(150)

tree.plot_tree(clf, filled=True,rounded=True)
fig.savefig('best_tree.png')
columns = list(X.columns)
#plt.show()